In [1]:
import pandas as pd
from ast import literal_eval
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.spatial import distance
from itertools import product
from adjustText import adjust_text
from matplotlib.patches import Patch
import matplotlib.ticker as mtick
from matplotlib.lines import Line2D

In [2]:
cntry_coll_fund='../../data/nf_folder/Data/DerivedData/Derived/coll_fund/cntry_coll_fund.csv'
cntry_cnt_path='../../data/nf_folder/Data/DerivedData/Derived/cntry_year_cnt.csv'
pubs_path='../../data/nf_folder/Data/DerivedData/CleanedRawData/pubs_author_funder.csv'
nindex_path="../../data/nf_folder/Data/AdditionalData/2021-NatureIndex.csv"

In [3]:
cntry_pub=pd.read_csv(cntry_coll_fund)
cntry_pub['funder_distinct']=cntry_pub['funder_distinct'].apply(
    lambda x:literal_eval(x) if x!='Not-Funded' else x)
cntry_pub.head()

,id,author_distinct,funder_distinct,year,rel_citation,IntCol,fund
0,30507292,Turkey,[Turkey],2009,0.485,notcol,single
1,30574690,United States,[United States],2009,0.703,col,single
2,30574690,France,[United States],2009,0.703,col,others
3,30574691,Russia,[Russia],2009,0.312,notcol,single
4,30585095,Spain,[Spain],2009,0.137,notcol,single


In [4]:
cntry_pub=cntry_pub.rename(columns={'author_distinct':'acntry'})

In [5]:
pubs_df=pd.read_csv(pubs_path)
pubs_df.head()

/home/miaoli/conda/envs/funding/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,author_country,funder_country,discipline,year,rel_citation,citation
0,30507292,['Turkey'],['Turkey'],77,2009,0.485,6
1,30574690,"['United States', 'France']","['United States', 'United States', 'United Sta...",83,2009,0.703,9
2,30574691,['Russia'],['Russia'],83,2009,0.312,4
3,30585095,"['Spain', 'Spain']","['Spain', 'Spain', 'Spain', 'Spain', 'Spain', ...",89,2009,0.137,1
4,30585097,"['Germany', 'Spain']",['Spain'],89,2009,0.410,3


In [6]:
pubs_df=pubs_df[['id','author_country']]
pubs_df['author_country']=pubs_df['author_country'].apply(lambda x:literal_eval(x))
pubs_df['author_distinct']=pubs_df['author_country'].apply(lambda x:list(set(x)))

In [7]:
cntry_pub=cntry_pub.merge(pubs_df[['id','author_distinct']],on='id',how='left')

In [8]:
cntry_pub.head()

,id,acntry,funder_distinct,year,rel_citation,IntCol,fund,author_distinct
0,30507292,Turkey,[Turkey],2009,0.485,notcol,single,[Turkey]
1,30574690,United States,[United States],2009,0.703,col,single,"[United States, France]"
2,30574690,France,[United States],2009,0.703,col,others,"[United States, France]"
3,30574691,Russia,[Russia],2009,0.312,notcol,single,[Russia]
4,30585095,Spain,[Spain],2009,0.137,notcol,single,[Spain]


In [9]:
def top(acntry,cntrys):
    for a in cntrys:
        if (a != acntry) and (a in tops):
            return 1 
    return 0

In [14]:
cntry_pub['top']=cntry_pub[['acntry','author_distinct']].apply(lambda x:top(x.acntry,x.author_distinct),axis=1)

In [11]:
nindex = pd.read_csv(nindex_path)

In [12]:
nindex.head(2)

,#,Country/territory,wos
0,1,United States of America (USA),United States
1,2,China,China


In [13]:
tops=nindex[:20].wos.values

In [16]:
cntry_pub.head(3)

,id,acntry,funder_distinct,year,rel_citation,IntCol,fund,author_distinct,top
0,30507292,Turkey,[Turkey],2009,0.485,notcol,single,[Turkey],0
1,30574690,United States,[United States],2009,0.703,col,single,"[United States, France]",1
2,30574690,France,[United States],2009,0.703,col,others,"[United States, France]",1


In [17]:
col_ratio=cntry_pub.groupby(
    ['acntry','year','IntCol']).size().reset_index(name='count')
col_ratio=col_ratio.pivot_table(index=['acntry','year'],columns='IntCol',values='count')
col_ratio=col_ratio.div(col_ratio.sum(axis=1), axis=0).reset_index()
col_ratio.head()

IntCol,acntry,year,col,notcol
0,Afghanistan,2009,0.823529,0.176471
1,Afghanistan,2010,0.909091,0.090909
2,Afghanistan,2011,0.928571,0.071429
3,Afghanistan,2012,0.897436,0.102564
4,Afghanistan,2013,0.925926,0.074074


In [18]:
def strc_df(data,indcol,col,start,end):
    def filter_df(data,indcol,col,year):
        df_filter=data[data.year==year][[indcol,col]]
        colnew=col+"_"+str(year)
        df_filter=df_filter.rename(columns={col:colnew})
        return df_filter
    df_start=filter_df(data,indcol,col,start)
    df_end=filter_df(data,indcol,col,end)
    df_meta=df_start.merge(df_end,on=indcol)
    return df_meta

In [19]:
col_ratio=strc_df(col_ratio,'acntry','col',2009,2018)

In [20]:
col_ratio.head(2)

IntCol,acntry,col_2009,col_2018
0,Afghanistan,0.823529,0.938462
1,Albania,0.671875,0.903553


In [54]:
cntry_pub.head()

,id,acntry,funder_distinct,year,rel_citation,IntCol,fund,author_distinct,top
0,30507292,Turkey,[Turkey],2009,0.485,notcol,single,[Turkey],0
1,30574690,United States,[United States],2009,0.703,col,single,"[France, United States]",1
2,30574690,France,[United States],2009,0.703,col,others,"[France, United States]",1
3,30574691,Russia,[Russia],2009,0.312,notcol,single,[Russia],0
4,30585095,Spain,[Spain],2009,0.137,notcol,single,[Spain],0


In [32]:
def get_fund_ratio(data,ind_top,colname):
    fratio=data[data.top==ind_top]
    fratio=fratio.groupby(
        ['acntry','year','fund']).size().reset_index(name='count')
    fratio=fratio.pivot_table(index=['acntry','year'],columns='fund',values='count',fill_value=0)
    fratio=fratio.div(fratio.sum(axis=1), axis=0).reset_index()
    fratio[colname]=fratio['cofund']+fratio['single']
    fratio=strc_df(fratio,'acntry',colname,2009,2018)
    return fratio

In [33]:
col_df=cntry_pub[cntry_pub.IntCol=='col']
ftop=get_fund_ratio(col_df,1,'fund_top')
fntop=get_fund_ratio(col_df,0,'fund_notop')

In [36]:
ftop[ftop.acntry=='United States']

fund,acntry,fund_top_2009,fund_top_2018
183,United States,0.422707,0.444564


In [37]:
fntop[fntop.acntry=='United States']

fund,acntry,fund_notop_2009,fund_notop_2018
159,United States,0.400077,0.416299


In [83]:
test = col_df[col_df.top==0].groupby(['acntry','year','fund']).size().reset_index(name='count')
test = test.pivot_table(index=['acntry','year'],columns='fund',values='count',fill_value=0)
#test=test.div(test.sum(axis=1), axis=0).reset_index()
#test['r']=test['cofund']+test['single']

In [78]:
s='China'
test.loc[s,:]

fund,cofund,not-identified,others,single
year,,,,
2009,5904,7803,4952,8712
2010,8423,6754,6414,10343
2011,10617,6416,7583,12795
2012,12187,6672,8297,15926
2013,14350,7504,9376,20003
2014,15966,8245,10171,24895
2015,18690,8492,10933,29894
2016,20756,10310,12385,34711
2017,22689,9873,13600,40701


In [82]:
test.loc[s,:]

fund,cofund,not-identified,others,single
year,,,,
2009,5594,7376,4764,8103
2010,8003,6334,6173,9623
2011,10047,5960,7310,11838
2012,11574,6190,7957,14731
2013,13521,6941,8991,18330
2014,15134,7504,9698,22710
2015,17614,7746,10443,27423
2016,19562,9288,11824,31640
2017,21414,8701,12935,36750


In [84]:

test.loc[s,:]

fund,cofund,not-identified,others,single
year,,,,
2009,310,427,188,609
2010,420,420,241,720
2011,570,456,273,957
2012,613,482,340,1195
2013,829,563,385,1673
2014,832,741,473,2185
2015,1076,746,490,2471
2016,1194,1022,561,3071
2017,1275,1172,665,3951


In [56]:
topn=['United States','China','Japan','South Korea','Germany','India','Canada','Spain','United Kingdom']

In [57]:
bins=np.arange(0,11)/10
labels=np.arange(0,10)/10
labels

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [59]:
meta=fund_ratio.merge(col_ratio,on='acntry',how='inner')
meta['x1']=pd.cut(meta['col_2009'],bins,labels=labels,include_lowest=True)
meta['y1']=pd.cut(meta['involved_2009'],bins,labels=labels,include_lowest=True)
meta.head()

,acntry,involved_2009,involved_2018,col_2009,col_2018,x1,y1
0,Afghanistan,0.000000,0.000000,0.823529,0.938462,0.8,0.0
1,Albania,0.000000,0.000000,0.671875,0.903553,0.6,0.0
2,Algeria,0.045139,0.091620,0.612863,0.604925,0.6,0.0
3,Andorra,0.000000,0.250000,0.500000,1.000000,0.4,0.0
4,Angola,0.000000,0.052632,0.966667,0.983607,0.9,0.0


In [61]:
meta_topn=meta[meta.acntry.isin(topn)]
meta_topn

,acntry,involved_2009,involved_2018,col_2009,col_2018,x1,y1
32,Canada,0.346941,0.368722,0.471687,0.614118,0.4,0.3
37,China,0.530131,0.737492,0.223630,0.265053,0.2,0.5
66,Germany,0.366843,0.452736,0.496058,0.618416,0.4,0.3
79,India,0.255829,0.325054,0.190892,0.270998,0.1,0.2
88,Japan,0.330938,0.429113,0.253929,0.349773,0.2,0.3
159,South Korea,0.450047,0.541698,0.254447,0.315588,0.2,0.4
160,Spain,0.478047,0.560032,0.434201,0.597677,0.4,0.4
182,United Kingdom,0.332168,0.413980,0.518897,0.701066,0.5,0.3
183,United States,0.422707,0.444564,0.321063,0.464636,0.3,0.4
